# Obteniendo datos de la API de Spotify

In [61]:
import base64
import requests
import pandas as pd

## Autorización

In [148]:
# función de autenticación de spotify

def spoty_aut (client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    credenciales = client_id + ':' + client_secret
    authorization = "Basic " + base64.b64encode(credenciales.encode()).decode()
    grant_type= 'client_credentials'
    resultado = requests.post(url, data={'grant_type':grant_type},
              headers={'Authorization':authorization, 'Content-Type':'application/x-www-form-urlencoded'})
    return resultado.json()

In [5]:
client_id = '59b694fd5440457186f976c5d2c33228'
client_secret = '2a76dafc52db47c2bf536e5382f3b0a3'

In [150]:
spoty_aut(client_id, client_secret)

{'access_token': 'BQDLs4Y3iv6ML24B585oHSEoKdC0IQReReraHO2kidpnYKSFGe8ziUGVlelf_UlTJ5Go93ehAceG6qW2Qqw',
 'token_type': 'Bearer',
 'expires_in': 3600}

## Obteniendo playlist

In [147]:
#función que crea df con variables necesarias para sherpa a partir de playlist_id

def df_sherpa(playlist_id):
    base = "https://api.spotify.com/v1/"
    playlist = requests.get(base+f"playlists/{playlist_id}/tracks",
                      headers = {'Authorization':f'''Bearer {resultado.json()['access_token']}''', 'Content-Type': 'application/json'})
    
    n_canciones=len(playlist.json()['items'])
    
    lista_df_tracks=[]
    
    for i in range(0,n_canciones):
        
        #obteniendo autor, nombre de canción y sus IDs de las canciones en la playlist
        artist_track=playlist.json()['items'][i]
        
        artist_id= artist_track['track']['artists'][0]['id']
        artist = artist_track['track']['artists'][0]['name']
        track = artist_track['track']['name']
        track_id = artist_track['track']['id']
        
        #obteniendo características de canciones de playlist
        audio_features = requests.get(base+f"audio-features/{track_id}",
                          headers = {'Authorization':f'''Bearer {resultado.json()['access_token']}''', 'Content-Type': 'application/json'})
        tempo = audio_features.json()['tempo']
        energy = audio_features.json()['energy']
        danceability = audio_features.json()['danceability']
        valence = audio_features.json()['valence']
        key = audio_features.json()['key']
        time_signature = audio_features.json()['time_signature']
        acousticness = audio_features.json()['acousticness']
        instrumentalness = audio_features.json()['instrumentalness']
        mode = audio_features.json()['mode']
    
        lista_track = [track_id, artist, track, tempo, energy, danceability, 
                 valence, key, time_signature, acousticness, instrumentalness, mode]
        lista_df_tracks.append(lista_track)
    
    df_sherpa=pd.DataFrame(lista_df_tracks, columns=["track_id", "artist", "track",  "tempo", "energy", "danceability", 
             "valence", "key", "time_signature", "acousticness", "instrumentalness", "mode"])
    return df_sherpa
    
    

In [146]:
df_sherpa("6doANzWu81vKWBsIiMLrHj")

,track_id,artist,track,tempo,energy,danceability,valence,key,time_signature,acousticness,instrumentalness,mode
0,4bP5n83Cx3Ek2JspqI3qXR,Aphex Twin,diskhat ALL prepared1mixed 13,110.010,0.539,0.852,0.3400,3,4,0.5180,0.865,1
1,0WTr0gFrQhKKAlxcaWTeRe,Gabriel Prokofiev,Voyage V: Fanta®,125.899,0.350,0.742,0.2280,0,4,0.3800,0.736,0
2,5f8jfumCm1ESZnoDvoGqlA,Jaubi,Lahore State of Mind - Remix,91.047,0.550,0.707,0.3340,6,4,0.1110,0.825,1
3,1LWDM0ldyMWfUe9jSmeuAm,Don Cherry,Benoego,88.025,0.567,0.648,0.8350,7,4,0.1940,0.663,1
4,29AYtOXVZscO66KeeGkjK0,James Holden,Pass Through The Fire / Bouri Bouri Manandabo,101.170,0.625,0.473,0.9250,8,4,0.0704,0.908,1
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1t8DAe37WZUVT4UppCAeZm,Kerala Dust,Spiegel,114.989,0.675,0.800,0.5770,1,4,0.0521,0.895,1
96,2YJ5LBrsDGXza8Gak2KBKQ,Landhouse,Back From the Black Forest,100.031,0.530,0.807,0.6660,1,4,0.2160,0.881,1
97,1kETMAlYAqu5nNr1c8ld4n,Forest Swords,Panic,130.016,0.428,0.489,0.0526,1,4,0.4780,0.767,1
98,57p1UwuJ19XMaC7gF9B52h,Mount Kimbie,Blind Night Errand,134.018,0.479,0.782,0.3150,8,4,0.0202,0.868,0


Algo a explorar al futuro:
audio_full_analysis = requests.get(base+f"audio-analysis/{track_id}",
                    headers = {'Authorization':f'''Bearer {resultado.json()['access_token']}''', 'Content-Type':
                    'application/json'})